# Train doc2vec

Load dependencies.

In [154]:
import glob
import random
import os
from sklearn.model_selection import train_test_split
import numpy as np
from contextlib import contextmanager
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from timeit import default_timer as timer
import multiprocessing as mp
from datetime import timedelta
import pandas as pd
from scipy.spatial.distance import cosine as cosine_distance

import logging
logging.basicConfig(level=logging.INFO)

## 1. Prepare training corpus 

Since we will be training a model with hyperparameters that need to be tuned, we need a validation set along with a test set.

- **Training Dataset**: The sample of data used to fit the model.
- **Validation Dataset**: The sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters.
- **Test Dataset**: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.

![image.png](attachment:image.png)
Image source: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7

Usually the Test set is curated to provide the gold standard, the carefully sampled data that spans the various classes that the model would face, when used in the real world. As with Kaggle, the final test set is released to the public usually only when a contest is nearing the end. However, here it is not curated, but randomly selected. It would be good in the future to have an astrophysicist carefully curate this test set to select a triplet set to test how well the model is able to detect the relationships within.

Here we randomly split our dataset into 2 — Train and Test — using the common 80:20 ratio. We set aside the Test set. We then randomly split the Train set, again using the 80:20 ratio, into the actual Train and Validation sets. We will iteratively train the model and validate it on these two sets. Then we will test the final model on the Test set. 

*Note: There is a [discussion](https://towardsdatascience.com/finally-why-we-use-an-80-20-split-for-training-and-test-data-plus-an-alternative-method-oh-yes-edc77e96295d) on how to choose best ratio. Here we just go with the conventional split, which is 80:20. But for my dissertation it may be worth looking into alternative ratios such as the scaling law, although oftentimes the increase in performance is small.*

In [2]:
# Collect all filenames
filenames = np.array(glob.glob('/Volumes/BRIENNAKH/Thesis/data/2020_06_09_abstract_tokens/*.npy'))
print('Number of abstracts: ' + str(len(filenames)))

# Randomly shuffle
print('Randomly shuffling...')
np.random.shuffle(filenames)

# Split 80:20 (we will do this part iteratively later in cross-validation)
print('Splitting files into two subsets with a ratio of 80:20...')
train_data, test_data = train_test_split(filenames, test_size=0.2)
train_data, validation_data = train_test_split(train_data, test_size=0.2)
print('Number of abstracts for training: ' + str(len(train_data)))
print('Number of abstracts for testing: ' + str(len(test_data)))
print('Number of abstracts for validation: ' + str(len(validation_data)))

Number of abstracts: 159616
Randomly shuffling...
Splitting files into two subsets with a ratio of 80:20...
Number of abstracts for training: 102153
Number of abstracts for testing: 31924
Number of abstracts for validation: 25539


## 2. Train the model

To train the model, our corpus can be simply a list of documents, but since we have a large corpus, we are using an iterable that streams the documents directly from disk/network. 

In [3]:
class DocumentIterator(object):
    def load_documents(self):
        for filename in self.filenames: 
            loaded_file = np.load(filename)
            tag = os.path.splitext(os.path.basename(filename))[0]
            abstract = TaggedDocument(words=loaded_file, tags=[tag])
            try:
                yield abstract
            except Exception as e:
                print('Error!' + str(e))
            finally:
                del loaded_file
                # print('Closed ' + tag)
                
    def __init__(self, filenames):
        self.filenames = filenames
        self.documents = self.load_documents()

    def __iter__(self):
        self.generator = self.load_documents() # Reset the iterator
        return self
    
    def __next__(self):
        abstract = next(self.documents)
        return abstract

Train the model.

Parameters: 
- dm = 0 means "distributed bag of words" (PV-DBOW). dm = 1 means "distributed memory" (PV-DM)

Notes: 
- The vocabulary is a dictionary (accessible via model.wv.vocab) of all of the unique words extracted from the training corpus along with the count (e.g., model.wv.vocab['penalty'].count for counts for the word penalty).

In [5]:
start = timer()
model = Doc2Vec(DocumentIterator(train_data),
                vector_size=300,
                alpha=0.025,
                min_alpha=0.001, 
                min_count=1,
                workers=mp.cpu_count(),
                dm=1,
                epochs=40)
#model.save('d2v.model')
print('Model Saved')
end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #10000, processed 2130028 words (16539/s), 30114 word types, 10000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #20000, processed 4258194 words (15906/s), 40930 word types, 20000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #30000, processed 6400029 words (17016/s), 49048 word types, 30000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #40000, processed 8525631 words (15496/s), 55860 word types, 40000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #50000, processed 10658562 words (15244/s), 61713 word types, 50000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #60000, processed 12781166 words (14471/s), 66916 word types, 60000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #70000, processed 14911706 words (14854/s), 71647 word types, 70000

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 8 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 9 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thr

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 17 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 18 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker t

INFO:gensim.models.base_any2vec:EPOCH - 26 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 27 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 36 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 37 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
INFO:gensim.models.base_any2vec:worker t

Model Saved
Execution time HH:MM:SS: 0:24:24.595717


## 3. Evaluate the model using triplets

From the remaining 20% papers, construct a dataset of triplets in which we randomly select 
- an anchor paper A, 
- a positively-related paper B which shares at least one author-assigned sub-subject with paper A, 
- and a negatively-related paper C which shares no sub-subjects with paper A. 

We expect papers A and B to have more similar vector representations than papers A and C, thus we will determine model accuracy as the total number of triplets where the similarity is higher between the anchor and positively related papers rather than between the anchor and negatively related papers:

We are using the validation set to evaluate the model. Load metadata for the validation set.

In [188]:
metadata = pd.read_csv('../data/2020_03_06_arxiv_metadata_astroph/arxiv_metadata_astroph.csv',
                       dtype={'filename': str,
                              'filename_parsed': str,
                              'identifier': str,
                              'updated': str,
                              'doi': str}, 
                       parse_dates=['date_retrieved'])

validation_ids = [os.path.splitext(os.path.basename(x))[0] for x in validation_data]
validation_metadata = metadata[metadata['filename_parsed'].isin(validation_ids)]
validation_metadata

,identifier,filename,spec,title,datestamp,created,updated,authors,categories,journal,doi,abstract,comments,date_retrieved,filename_parsed
19924,oai:arXiv.org:0901.0007,0901.0007,physics:astro-ph,"Eternal Inflation, Bubble Collisions, and the ...",2009-09-28,2009-01-02,2009-06-04,"Ben Freivogel, Matthew Kleban, Alberto Nicolis...",hep-th astro-ph.CO gr-qc,"JCAP 0908:036,2009",10.1088/1475-7516/2009/08/036,We compute the probability distribution for ...,NaN,2020-03-07 02:10:36.300743,0901.0007
19956,oai:arXiv.org:0901.0202,0901.0202,physics:astro-ph,Modeling the RV and BVS of active stars,2015-05-13,2009-01-01,NaN,"Cezary Migaszewski, Grzegorz Nowak",astro-ph.SR,NaN,10.1051/eas/1042015,We present a method of modeling the radial v...,NaN,2020-03-07 02:10:36.300743,0901.0202
19957,oai:arXiv.org:0901.0215,0901.0215,physics:astro-ph,The universe as a black hole in isotropic coor...,2009-01-21,2009-01-02,NaN,Nikodem J. Poplawski,gr-qc astro-ph.CO,NaN,NaN,We show that the radial geodesic motion of a...,NaN,2020-03-07 02:10:36.300743,0901.0215
19958,oai:arXiv.org:0901.0223,0901.0223,physics:astro-ph,NLTE models of line-driven stellar winds III. ...,2009-11-13,2009-01-02,NaN,"Jiri Krticka, Jiri Kubat",astro-ph.SR,NaN,10.1111/j.1365-2966.2009.14457.x,We study the influence of X-rays on the wind...,NaN,2020-03-07 02:10:36.300743,0901.0223
19959,oai:arXiv.org:0901.0239,0901.0239,physics:astro-ph,Deep-Sea Acoustic Neutrino Detection and the A...,2019-08-13,2009-01-02,2009-04-08,Robert Lahmann,astro-ph.IM,NaN,10.1016/j.nima.2008.12.204,The use of conventional neutrino telescope m...,NaN,2020-03-07 02:10:36.300743,0901.0239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179524,oai:arXiv.org:1912.13197,1912.13197,physics:astro-ph,Nonlinear Coupling of Electromagnetic and Elec...,2020-02-19,2019-12-31,NaN,"N. L. Shatashvili, S. M. Mahajan, V. I. Berezh...",physics.plasm-ph astro-ph.SR,NaN,10.1063/1.5135959,Nonlinear wave--coupling is studied in a mul...,NaN,2020-03-07 02:10:36.300743,1912.13197
179531,oai:arXiv.org:1912.13240,1912.13240,physics:astro-ph,ALMA observations reveal no preferred outflow-...,2020-02-19,2019-12-31,NaN,"T. Baug, Ke Wang, Tie Liu, Mengyao Tang, Qizho...",astro-ph.GA,NaN,10.3847/1538-4357/ab66b6,We present a statistical study on the orient...,NaN,2020-03-07 02:10:36.300743,1912.13240
179534,oai:arXiv.org:1912.13326,1912.13326,physics:astro-ph,EFT Compatible PBHs: Effective Spawning of the...,2020-01-15,2019-12-31,NaN,"Amjad Ashoorioon, Abasalt Rostami, Javad T. Fi...",astro-ph.CO gr-qc hep-ph hep-th,NaN,NaN,Most of the inflationary scenarios that try ...,NaN,2020-03-07 02:10:36.300743,1912.13326
179549,oai:arXiv.org:2001.00085,2001.00085,physics:astro-ph,How surfaces shape the climate of habitable ex...,2020-02-19,2019-12-31,2020-02-06,"Jack Madden, Lisa Kaltenegger",astro-ph.EP astro-ph.SR,NaN,10.1093/mnras/staa387,Large ground- and space-based telescopes wil...,NaN,2020-03-07 02:10:36.300743,2001.00085


Set categories.

In [283]:
acceptable_categories = ['astro-ph.SR',
                         'astro-ph.CO',
                         'astro-ph.GA',
                         'astro-ph.HE',
                         'astro-ph.EP',
                         'astro-ph.IM']

Construct triplets within the validation set. 

In [393]:
def generate_triplet():
    '''
    Randomly selects an anchor paper A, 
    a positively-related paper B which shares at least one category with paper A,
    and a negatively-related paper C which shares no sub-subjects with paper A. 
    
    Returns a tuple (A, B, C) containing each paper's arXiv ID/filename.
    '''
    
    # Select Paper A
    a_id = random.choice(data['filename_parsed'].tolist())
    a = data[data['filename_parsed'] == a_id]
    a_categories = [x for x in a['categories'].iloc[0].split(' ') if x in acceptable_categories]
    data.drop(a.index, inplace=True) # Remove A from data
    print('\nRandomly chose Paper A from ' + str((len(data))) + ' eligible papers...')
    print('Paper A: ' + a_id)
    print('Categories: ' + str(a_categories))
    
    # Select Paper B
    b_set = data[data['categories'].str.split(' ').apply(lambda x: np.any(np.isin(x, a_categories)))]
    b_id = random.choice(b_set['filename_parsed'].tolist())
    b = data[data['filename_parsed'] == b_id]
    b_categories = [x for x in b['categories'].iloc[0].split(' ') if x in acceptable_categories]
    print('Randomly chose Paper B from ' + str(len(b_set)) + ' eligible papers...')
    print('Paper B: ' + b_id)
    print('Categories: ' + str(b_categories))
    
    # Select Paper C
    c_set = data.merge(b_set, how='outer', indicator=True).loc[lambda x: x['_merge']=='left_only'] 
    c_id = random.choice(c_set['filename_parsed'].tolist())
    c = data[data['filename_parsed'] == c_id]
    c_categories = [x for x in c['categories'].iloc[0].split(' ') if x in acceptable_categories]
    print('Randomly chose Paper C from ' + str(len(c_set)) + ' eligible papers...')
    print('Paper C: ' + c_id)
    print('Categories: ' + str(c_categories))
    
    print(len(b_set) + len(c_set) == len(data))
    data.drop([b.index[0], c.index[0]], inplace=True)
    
    return (a, b, c)

def get_vector(doc):
    '''Takes a Pandas row for a paper.'''
    loaded = np.load('../data/2020_06_09_abstract_tokens/' + doc['filename_parsed'].iloc[0] + '.npy')
    return model.infer_vector(loaded)

def calculate_accuracy(triplet):
    '''Calculates accuracy of one given triplet.'''
    # Calculate cosine distances
    a_vector = get_vector(triplet[0])
    b_vector = get_vector(triplet[1])
    c_vector = get_vector(triplet[2])
    ab_dist = cosine_distance(a_vector, b_vector)
    ac_dist = cosine_distance(a_vector, c_vector)
    
    # Calculate accuracy
    if ab_dist > ac_dist:
        return 1
    else:
        return 0
    
def evaluate(triplets):
    '''Calculates averaged accuracy of all given triplets.'''
    accuracy = 0
    for triplet in triplets:
        accuracy += calculate_accuracy(triplet)
    return accuracy/len(triplets)

In [387]:
data = metadata[metadata['filename_parsed'].isin(validation_ids)]
triplets = []

# Generage 10,000 triplets
for i in range(0, 10000):
    triplet = generate_triplet()
    triplets.append(triplet)


Randomly chose Paper A from 25538 eligible papers...
Paper A: 1505.05539
Categories: ['astro-ph.EP']
Randomly chose Paper B from 2770 eligible papers...
Paper B: 1408.6807
Categories: ['astro-ph.EP']
Randomly chose Paper C from 22768 eligible papers...
Paper C: 1605.06136
Categories: ['astro-ph.GA', 'astro-ph.SR']
True

Randomly chose Paper A from 25535 eligible papers...
Paper A: 1002.4596
Categories: ['astro-ph.EP']
Randomly chose Paper B from 2768 eligible papers...
Paper B: 1501.00957
Categories: ['astro-ph.EP']
Randomly chose Paper C from 22767 eligible papers...
Paper C: 1108.1415
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 25532 eligible papers...
Paper A: 1605.05321
Categories: ['astro-ph.SR', 'astro-ph.HE']
Randomly chose Paper B from 11677 eligible papers...
Paper B: 1701.01116
Categories: ['astro-ph.HE']
Randomly chose Paper C from 13855 eligible papers...
Paper C: 1303.4519
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 25529 eligible paper

Randomly chose Paper B from 5860 eligible papers...
Paper B: 1007.5082
Categories: ['astro-ph.HE', 'astro-ph.CO']
Randomly chose Paper C from 19600 eligible papers...
Paper C: 1406.2209
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 25457 eligible papers...
Paper A: 1506.01496
Categories: ['astro-ph.GA']
Randomly chose Paper B from 6305 eligible papers...
Paper B: 1108.0205
Categories: ['astro-ph.GA', 'astro-ph.CO']
Randomly chose Paper C from 19152 eligible papers...
Paper C: 0910.3565
Categories: ['astro-ph.HE', 'astro-ph.CO']
True

Randomly chose Paper A from 25454 eligible papers...
Paper A: 1312.5310
Categories: ['astro-ph.CO']
Randomly chose Paper B from 7863 eligible papers...
Paper B: 1005.3101
Categories: ['astro-ph.CO']
Randomly chose Paper C from 17591 eligible papers...
Paper C: 1401.0244
Categories: ['astro-ph.SR']
True

Randomly chose Paper A from 25451 eligible papers...
Paper A: 1911.02012
Categories: ['astro-ph.EP', 'astro-ph.SR']
Randomly chose Paper B 

Randomly chose Paper B from 2674 eligible papers...
Paper B: 1409.0458
Categories: ['astro-ph.SR', 'astro-ph.IM']
Randomly chose Paper C from 22708 eligible papers...
Paper C: 1910.06970
Categories: ['astro-ph.GA', 'astro-ph.CO']
True

Randomly chose Paper A from 25379 eligible papers...
Paper A: 1210.4602
Categories: ['astro-ph.GA', 'astro-ph.CO']
Randomly chose Paper B from 12732 eligible papers...
Paper B: 1610.06595
Categories: ['astro-ph.GA']
Randomly chose Paper C from 12647 eligible papers...
Paper C: 1001.4209
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 25376 eligible papers...
Paper A: 1110.3312
Categories: ['astro-ph.CO', 'astro-ph.GA', 'astro-ph.HE']
Randomly chose Paper B from 17289 eligible papers...
Paper B: 1505.03839
Categories: ['astro-ph.CO']
Randomly chose Paper C from 8087 eligible papers...
Paper C: 1505.03704
Categories: ['astro-ph.EP']
True

Randomly chose Paper A from 25373 eligible papers...
Paper A: 1102.3696
Categories: ['astro-ph.SR']
Rando

Randomly chose Paper B from 6442 eligible papers...
Paper B: 1303.3417
Categories: ['astro-ph.SR']
Randomly chose Paper C from 18862 eligible papers...
Paper C: 1808.05613
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 25301 eligible papers...
Paper A: 1409.1572
Categories: ['astro-ph.HE']
Randomly chose Paper B from 5813 eligible papers...
Paper B: 0912.3596
Categories: ['astro-ph.HE']
Randomly chose Paper C from 19488 eligible papers...
Paper C: 1701.08161
Categories: ['astro-ph.EP']
True

Randomly chose Paper A from 25298 eligible papers...
Paper A: 1104.5470
Categories: ['astro-ph.CO']
Randomly chose Paper B from 7814 eligible papers...
Paper B: 1612.08919
Categories: ['astro-ph.GA', 'astro-ph.CO']
Randomly chose Paper C from 17484 eligible papers...
Paper C: 1305.7463
Categories: ['astro-ph.GA']
True

Randomly chose Paper A from 25295 eligible papers...
Paper A: 1906.07200
Categories: ['astro-ph.HE']
Randomly chose Paper B from 5811 eligible papers...
Paper B: 1708.

Randomly chose Paper B from 7789 eligible papers...
Paper B: 1509.03930
Categories: ['astro-ph.CO']
Randomly chose Paper C from 17437 eligible papers...
Paper C: 1801.06075
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 25223 eligible papers...
Paper A: 0906.1783
Categories: ['astro-ph.SR']
Randomly chose Paper B from 6426 eligible papers...
Paper B: 1704.08363
Categories: ['astro-ph.GA', 'astro-ph.SR']
Randomly chose Paper C from 18797 eligible papers...
Paper C: 1602.01555
Categories: ['astro-ph.GA']
True

Randomly chose Paper A from 25220 eligible papers...
Paper A: 1407.7609
Categories: ['astro-ph.SR', 'astro-ph.GA']
Randomly chose Paper B from 11628 eligible papers...
Paper B: 1908.06113
Categories: ['astro-ph.SR', 'astro-ph.GA']
Randomly chose Paper C from 13592 eligible papers...
Paper C: 1711.05416
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 25217 eligible papers...
Paper A: 1003.0419
Categories: ['astro-ph.CO']
Randomly chose Paper B from 7787 

...

Randomly chose Paper C from 288 eligible papers...
Paper C: 1603.06631
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 365 eligible papers...
Paper A: 1910.02846
Categories: ['astro-ph.EP']
Randomly chose Paper B from 28 eligible papers...
Paper B: 1406.0210
Categories: ['astro-ph.SR', 'astro-ph.EP']
Randomly chose Paper C from 337 eligible papers...
Paper C: 1201.2987
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 362 eligible papers...
Paper A: 1801.05428
Categories: ['astro-ph.GA']
Randomly chose Paper B from 99 eligible papers...
Paper B: 1408.4558
Categories: ['astro-ph.GA']
Randomly chose Paper C from 263 eligible papers...
Paper C: 1912.09858
Categories: ['astro-ph.CO']
True

Randomly chose Paper A from 359 eligible papers...
Paper A: 1907.03768
Categories: ['astro-ph.HE']
Randomly chose Paper B from 78 eligible papers...
Paper B: 1405.4547
Categories: ['astro-ph.HE']
Randomly chose Paper C from 281 eligible papers...
Paper C: 1404.6102
Categories: [

Randomly chose Paper B from 74 eligible papers...
Paper B: 1706.01286
Categories: ['astro-ph.GA']
Randomly chose Paper C from 210 eligible papers...
Paper C: 1710.05688
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 281 eligible papers...
Paper A: 1611.07854
Categories: ['astro-ph.CO']
Randomly chose Paper B from 100 eligible papers...
Paper B: 1708.01253
Categories: ['astro-ph.CO']
Randomly chose Paper C from 181 eligible papers...
Paper C: 0903.0805
Categories: ['astro-ph.IM']
True

Randomly chose Paper A from 278 eligible papers...
Paper A: 1108.4520
Categories: ['astro-ph.CO']
Randomly chose Paper B from 98 eligible papers...
Paper B: 1501.05129
Categories: ['astro-ph.CO']
Randomly chose Paper C from 180 eligible papers...
Paper C: 1408.0734
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 275 eligible papers...
Paper A: 1103.2375
Categories: ['astro-ph.SR']
Randomly chose Paper B from 51 eligible papers...
Paper B: 1701.07353
Categories: ['astro-ph.SR']


Randomly chose Paper A from 197 eligible papers...
Paper A: 1503.05200
Categories: ['astro-ph.CO']
Randomly chose Paper B from 76 eligible papers...
Paper B: 1011.1151
Categories: ['astro-ph.CO']
Randomly chose Paper C from 121 eligible papers...
Paper C: 1506.04564
Categories: ['astro-ph.EP']
True

Randomly chose Paper A from 194 eligible papers...
Paper A: 1903.06194
Categories: ['astro-ph.CO']
Randomly chose Paper B from 74 eligible papers...
Paper B: 1209.4643
Categories: ['astro-ph.CO']
Randomly chose Paper C from 120 eligible papers...
Paper C: 1712.06912
Categories: ['astro-ph.GA']
True

Randomly chose Paper A from 191 eligible papers...
Paper A: 1405.3287
Categories: ['astro-ph.GA']
Randomly chose Paper B from 50 eligible papers...
Paper B: 1601.00937
Categories: ['astro-ph.GA']
Randomly chose Paper C from 141 eligible papers...
Paper C: 1509.06341
Categories: ['astro-ph.IM']
True

Randomly chose Paper A from 188 eligible papers...
Paper A: 1810.08852
Categories: ['astro-ph.HE

Randomly chose Paper C from 74 eligible papers...
Paper C: 1809.00259
Categories: ['astro-ph.HE']
True

Randomly chose Paper A from 101 eligible papers...
Paper A: 1109.2594
Categories: ['astro-ph.CO']
Randomly chose Paper B from 37 eligible papers...
Paper B: 1804.09660
Categories: ['astro-ph.CO']
Randomly chose Paper C from 64 eligible papers...
Paper C: 1411.1674
Categories: ['astro-ph.GA']
True

Randomly chose Paper A from 98 eligible papers...
Paper A: 1902.10868
Categories: ['astro-ph.GA']
Randomly chose Paper B from 27 eligible papers...
Paper B: 1303.7368
Categories: ['astro-ph.GA', 'astro-ph.CO']
Randomly chose Paper C from 71 eligible papers...
Paper C: 1411.0128
Categories: ['astro-ph.SR']
True

Randomly chose Paper A from 95 eligible papers...
Paper A: 1706.05872
Categories: ['astro-ph.HE']
Randomly chose Paper B from 17 eligible papers...
Paper B: 1011.0005
Categories: ['astro-ph.HE', 'astro-ph.SR']
Randomly chose Paper C from 78 eligible papers...
Paper C: 1411.2595
Categ

IndexError: Cannot choose from an empty sequence

In [ ]:
evaluate(triplets)

We can form a certain number of triplets for validation set. I think that I should be forming these triplets at the time I split the data, so I don't toss any papers. Idk if this would form a bias in the training dataset. Form a fixed number of triplets as the validation set, and the test set, then whatever's left is the train set. Also how would this work with cross validation? How do I split the data for that?